In [1]:
!pip install python_speech_features
!pip install scipy

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5888 sha256=b9d0bd4ec6e1205e7f939d934dee2efaeddcdc0465bdf13b8577ba532d1166b3
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [3]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

import os
import _pickle
import random
import operator

from tempfile import TemporaryFile

import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#function to get the distance between 2 feature vectors and find neighbours
def getNeighbors(trainingSet, instance, k):
  distances = []
  for x in range (len(trainingSet)):
    dist = distance(trainingSet[x], instance, k)+distance(instance, trainingSet[x], k)
    distances.append((trainingSet[x][2],dist))
  distances.sort(key=operator.itemgetter(1))
  neighbors = []
  for x in range(k):
    neighbors.append(distances[x][0])
  return neighbors

#identify the class of the neighbours
def nearestClass(neighbors):
  classVote = {}
  for x in range(len(neighbors)):
    response = neighbors[x]
    if response in classVote:
      classVote[response]+=1
    else:
      classVote[response] = 1
    
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

#function to evaluate the model
def getAccuracy(testSet, prediction):
  correct = 0
  for x in range(len(testSet)):
    if testSet[x][-1] == prediction[x]:
      correct+=1
    
  return (1.0*correct)/len(testSet)

directory = '/content/drive/MyDrive/deep-learning-music-genre-classification/Data/genres_original/'
f = open("my.dat", 'wb')

i=0
for folder in os.listdir(directory):
  i+=1
  if i==11:
    break
  for file in os.listdir(directory+folder):
    (rat, sig) = wav.read(directory+folder+"/"+file)
    mfcc_feat = mfcc(sig, rat, winlen = 0.020, appendEnergy = False)
    covariance = np.cov(np.matrix.transpose(mfcc_feat))
    mean_matrix = mfcc_feat.mean(0)
    feature = (mean_matrix, covariance, i)
    _pickle.dump(feature,f)

f.close()


dataset = []
def loadDataset(filename, split, trSet, teSet):
  with open("my.dat", 'rb') as f:
    while True:
      try:
        dataset.append(_pickle.load(f))
      except EOFError:
        f.close()
        break
  
  for x in range(len(dataset)):
    if random.random() < split:
      trSet.append(dataset[x])
    else:
      teSet.append(dataset[x])
  
trainingSet = []
testSet = []
loadDataset("my.dat",0.8,trainingSet,testSet)







In [ ]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

# making predictions using KNN
leng = len(testSet)
predictions = []
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

test_dir = "/content/drive/MyDrive/deep-learning-music-genre-classification/Test/"
# test_file = test_dir + "test.wav"
test_file = test_dir + "test5.wav"
# test_file = test_dir + "test4.wav"

(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)

from collections import defaultdict
results = defaultdict(int)

directory = "/content/drive/MyDrive/deep-learning-music-genre-classification/Data/genres_original/"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

pred = nearestClass(getNeighbors(dataset, feature, 5))
print(results[pred])

0.7580645161290323
country
